In [ ]:
'''
This notebook is used for running the ML model for chord prediction in Jazz.

Please see the gitHub repository https://github.com/ldriever/ML_Jazz/ for further information on the project and
further information about running this notebook.
'''

In [ ]:
# First import all of the necessary libraries and the helper functions from the file ML_core_functions.py
import torch
import numpy as np
import pandas as pd
import copy

from ML_core_functions import divide_data, run_model
from dataframe_helper import fill_melody_info

In [ ]:
# This dictionary allows the user of this program to set the desired options for running this ML model
# For one thing, this includes setting the hyperparameters, namely batch size, learning rate, LSTM hidden size,...
#   ... number of LSTM layers, embedding size, patience, and weight decay. The values provided here correspond...
#   ...to the values provided in the report (see GitHub). The weight decay and number of LSTM layers is automatically...
#   ...adjusted to whether or not the model is run with or without melody information.
# The dictionary furthermore specifies where the input data can be found and where the results are to be saved.
# Finally it is possible to specify how many folds are to be used for k-fold cross validation, and whether training...
#   ...should be done using a GPU (please only set training on a GPU to True if a GPU is indeed available on your device).

Params = {
    "batch_size": 64,
    "learning_rate": 0.014,
    "lstm_hidden_size": 128,
    "num_lstm_layers": 0,
    "embedding_size": 64,
    "patience": 30,
    "weight_decay": 0,
    "input_data_path_no_melody": 'data_array_without_melody.pt',
    "input_data_path_with_melody": 'data_array_with_melody.pt',
    "output_options_path": 'output_options.pt',
    "model_path_save": None,
    "data_path_save": "../output_data_array.csv",
    "train_on_gpu": False,
    "k-fold": 10
}

In [ ]:
# This cell loads the input data for models with and without melody information, and the output options.
# Please make sure that the corresponding files are indeed in the right locations

basic_data_array = torch.load(Params["input_data_path_no_melody"])
melody_data_array = torch.load(Params["input_data_path_with_melody"])
output_options = torch.load(Params["output_options_path"])

In [ ]:
# Shuffle the two data arrays, seeding before each shuffling ensures that the arrays have the melids in the same order

# For model without melody information (i.e. the baseline model):
np.random.seed(1) 
# Shuffle a copy, not the original array:
basic_shuffled_data = copy.deepcopy(basic_data_array) 
np.random.shuffle(basic_shuffled_data)
# Concatenate the data array to itself to allow for easier data segmentation for k-fold cross validation
cyclic_basic_data = np.hstack((basic_shuffled_data, basic_shuffled_data))

# For model with melody information
# Same steps as above, but for the data corresponding to the melody model
np.random.seed(1) 
melody_shuffled_data = copy.deepcopy(melody_data_array)
np.random.shuffle(melody_shuffled_data)
cyclic_melody_data = np.hstack((melody_shuffled_data, melody_shuffled_data))

In [ ]:
# Create a pandas data frame for storing all data
# In this cell the columns are created and both melid and target chords are already filled in
# Later in this file the two prediction columns are filled in. The remaining columns are filled in a different file

columns = ["melid", "bar", "beat", "target", "prediction_no_melody", "prediction_with_melody", "notes", "fold"]
df = pd.DataFrame(0, index=np.arange(0), columns=columns)

for i in range(len(basic_data_array)):
    df_local = pd.DataFrame(0, index=np.arange(len(basic_data_array[i]["target"])), columns=columns)
    df_local = df_local.assign(melid=basic_data_array[i]["melid"])
    df_local = df_local.assign(target=np.array(output_options)[basic_data_array[i]['target'] + 1])
    df = df.append(df_local, ignore_index=True)

In [ ]:
# This cell  trains a model without melody information and a model with melody information per iteration, for k times
# For each iteration, the data is split such that one of the k blocks is used for validation, one is used for...
#   ...testing, and the remaining ones are used for training.
# For more details, please consult the report.

for k in range(Params["k-fold"]):
    # First the model without melodies
    train_data, val_data, test_data = divide_data(cyclic_basic_data, k, Params["k-fold"], output_options)

    basic_test_predictions = run_model(train_data, val_data, test_data, Params, with_melody=False, counter=k)

    for i in range(len(basic_test_predictions)):
        df.loc[df["melid"] == int(basic_test_predictions[i][0]), 'prediction_no_melody'] = np.array(output_options)[basic_test_predictions[i][1]+np.ones(len(basic_test_predictions[i][1]), dtype=int)]
        df.loc[df["melid"] == int(basic_test_predictions[i][0]), 'fold'] = k+1

    print(f"Finished round {k + 1} for chords-only")

    # Then the model with melodies
    train_data, val_data, test_data = divide_data(cyclic_melody_data, k, Params["k-fold"], output_options)

    melody_test_predictions = run_model(train_data, val_data, test_data, Params, with_melody=True, counter=k)

    for i in range(len(basic_test_predictions)):
        df.loc[df["melid"] == int(melody_test_predictions[i][0]), 'prediction_with_melody'] = np.array(output_options)[melody_test_predictions[i][1]++np.ones(len(melody_test_predictions[i][1]), dtype=int)]

    print(f"Finished round {k + 1} for model with melodies")

    # After each fold of the cross-validation the results are stored to avoid loss of data in case problems should occur
    df.to_csv(Params["data_path_save"])


In [ ]:
# Then fill the columns for beat, bar, and notes
fill_melody_info(df, Params["data_path_save"])

In [ ]:
# If it is desired to load the results to analyze them, the following function can, for instance, be used
data = pd.read_csv(Params["data_path_save"])